<img src="https://jupyter.org/assets/main-logo.svg" heigt=60> <font size="14" color=grey>  DHI Campus - Jupyter Training</font>  
***

# Jupyter Demo 

in a practical DHI-project context.


## Task & Motivation

This is a project we conducted for the Port of Hamburg (Germany) which required simulations of 3D flow in an estuary with MIKE 3D FM based on an existing model which might require updating. 

The projects location, the existing model boundaries as well as measurement locations are illustrated in the **interactive map below**.

In [1]:
""" Create Dataframe with important coordinates (Points)"""
import pandas as pd
import geopandas as gpd
from shapely import geometry

df_meta = pd.read_excel(r'demobook_data\metainfo.xlsx')

# get only relevant information for pois and remove nans
df = df_meta[['POI Name', 'lat', 'lng']]
df = df[df[df.columns[1]].notna()]
# get only relevant information for waterlevel and remove nans
df_wl = df_meta[['Pegel', 'Easting', 'Northing']]
df_wl = df_wl[df_wl[df_wl.columns[1]].notna()]

# create a geometry column (required for geopandas)
df['geometry'] = [geometry.Point(x, y) for x, y in zip(df[df.columns[2]], 
                                                       df[df.columns[1]])]
# create a geometry column (required for geopandas)
df_wl['geometry'] = [geometry.Point(x, y) for x, y in zip(df_wl[df_wl.columns[1]], 
                                                       df_wl[df_wl.columns[2]])]

""" Create geodataframe from DataFrame """

# convert to geopandas geodataframe by specifying 
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="+init=epsg:4326")
gdf_wl = gpd.GeoDataFrame(df_wl, geometry='geometry', crs="+init=epsg:25832")
gdf_wl = gdf_wl.to_crs({'init': 'epsg:4326'})



""" Get Model Domain from shapefile """
shpf_model = r'demobook_data\OpCIS2018_Stadersand_modify_Elements01.shp'
gdf_model = gpd.read_file(shpf_model)



    
""" Create Map """
import folium
from folium.plugins import MeasureControl

# initialize folium map around centroid coordinates
fmap = folium.Map(location=[53.563,9.7545],zoom_start=11,
                  tiles='cartodbpositron',
                  prefer_canvas=True,
                  control_scale=True)

# add further background maps
folium.TileLayer('stamenterrain',attr='Stamen').add_to(fmap)
folium.TileLayer('openstreetmap',attr='OSM').add_to(fmap)



""" add Model domain to folium map """ 
opacity=.3
weight=.5 
fill_color='#fcba03'#'#51f542'

def style_function(feature):
        return {
            'fillOpacity': opacity,
            'weight': weight,
            'fillColor': fill_color,
        }
    
geojson = folium.GeoJson(gdf_model.geometry, 
                         name='Modellgebiet OpCIS',
                         style_function=style_function).add_to(fmap)



""" Add Points of intrest to map """
interest_points = folium.FeatureGroup(name='Relevante Punkte')
for ix, row in gdf.iterrows():
        folium.Marker(location=[row.geometry.y, row.geometry.x],
                            #radius=1,
                            popup=str(row[df.columns[0]]),
                            icon=folium.Icon(color='red', icon='info-sign')
                     ).add_to(interest_points)
interest_points.add_to(fmap)



""" Add Waterlevel measurements to map"""
wlmeas_points = folium.FeatureGroup(name='Pegelmessstellen')
for ix, row in gdf_wl.iterrows():
        folium.CircleMarker(location=[row.geometry.y, row.geometry.x],
                            radius=3,
                            popup=str(row[df_wl.columns[0]]),
                            color='#0062ff'
                     ).add_to(wlmeas_points)
wlmeas_points.add_to(fmap)


    
# add a bit of last functionality to map
folium.LayerControl().add_to(fmap)
folium.plugins.MeasureControl().add_to(fmap)
fmap

**The main (simplyfied) goals of this project were**:

1. To identify relevant hydrological conditions and resulting scenarios to simulate
2. Conduct simulation runs in MIKE
3. Analyse results (e.g. current speeds)

## Planning

The Gantt chart from the proposal is shown below to stay informed about the original plan.

<img src="demobook_data/GANTT_PLAN.PNG" alt="Projektplanung" style="width: 700px;"/>

# Track current project status

The current project status is illustrated here as a simple table where progress is tracked by checking the "done" column.

| Nr. [Chapter] | Task | Description | Data Source | Finished |
|-----|---------|--------------|--------|---------------|
|<b> 1 [4,5] <b>| <b> Datenbeschaffung <b>| Daten aus Modelluntersuchungen Este (proj: ) und Köhlbrand (proj: ) wiederverwenden  (KOS: ETRS_1998_UTM_Zone32N) | | |
| | Korngrößenverteilung im Parkhafen aktualisieren falls möglich. Zuordnung zu Messpunkten | Sedimentkarte WSA, Stand 2006 liegt vor. Wenn möglich aktualisieren| HPA |X | |
| | Schwebstoffmessungen im Bereich des Parkhafen | Messdaten aus 2014 liegen vor, zusätzlich Daten aus 2016 und 2018 (Kap.6.3.2 und 6.3.3) | HPA | X |
| | Peilungen im Bereich des Parkhafen | Peilungen zwischen April und August 2016 (Kap.6.1.1.2) | HPA | X |
| | (optional) Baggermengen und Zeitpunkte im Bereich des Parkhafen | Notwendigkeit Prüfen - könnten seitens HPA bereitgestellt werden | HPA | X |
| | Ggf. Informationen zu Sedimentbudgets in Hafenteilen | |Sule (HPA) | |
| | Trübungsmessungen | Rückschluss durch Trübungsmessungen auf mobile Sedimente | Portal Tideelbe (Institut für Hygiene un Umwelt Hamburg, Wasserstrassen- und Schifffahrtsamt Hamburg, Bundesanstalt für Gewässerkunde) | X |
| | Abfluss in Neu Darchau | Tägliche Werte zwischen 1903 und 2017 | Wasserstraßen- und Schifffahrtsamt Magdeburg, Portal Tideelbe  | X |
| | Wasserstand bei Stadersand | Minütliche Werte, 1997-2017 (Tidehub 11.1964 - 2018) | HPA, Portal Tideelbe | X |
| | Georeferenzierte und digitale Planunterlagen in ETRS_1998_UTM_Zone32N | | | |
| | Solltiefe und genaue Lage der Elbvertiefung für die Einbindung ins Modell | | | |
|-----|---------|--------------|--------|---------------|
|<b> 2 [6] <b>| <b> Analyse Oberwasserzufluss <b> | Sedimenttransport maßgeblich vom Oberwasserzufluss abhängig. Daher Pegel Neu Darchau analysieren und in 2 Klassen einteilen (z.B. Klasse 1: 350m³/s, Klasse 2: 900m³/s). | | |
| | Kalibrierung im Bereich der Abflussklasse 1 | | | |
| | Kalibrierung im Bereich der Abflussklasse 2 | | | |


# Preprocessing (aka Exploring Data)
Analysis of representative boundary conditions / time periods for simulation based on measurements.
To identify relevant hydrological conditions (Here: Discharge).


In [2]:
import pandas as pd
# Erstelle Dataframe mit allen Messstellen 
# TODO: Später als Datenbank mit Zeiträumen etc (was kann DHI WaterData da schon?)
mess_infos = pd.DataFrame(columns=['Station','X','Y','X2','Y2','Stromkilometer','Z','Messung', 'Zeitschritt [min]','Zeitraum Start','Zeitraum Ende','erhebende Organisation'],
                         data = 
                          [['Bunthaus', 10.063998678408677 , 53.46140954926461, 'NaN', 'NaN', 609.8, 'NaN', 'Wasserstand [mNN]', '1','1997-11-01','2018-01-31', 'Hamburg Port Authority (HPA)'],
                           ['Schoepfstelle', 10.06157347792936, 53.50839861601538, 'NaN','NaN', 615.3, 'NaN', 'Wasserstand [mNN]', '1','1999-11-01','2018-01-31', 'Hamburg Port Authority (HPA)'],
                           ['Harburg', 9.99179527656458, 53.47272388834046, 'NaN','NaN', 616.4, 'NaN', 'Wasserstand [mNN]', '1','1999-11-01','2018-01-31', 'Hamburg Port Authority (HPA)'],
                           ['St.Pauli', 9.969996726842327, 53.54568502657209, 'NaN','NaN', 623, 'NaN', 'Wasserstand [mNN]', '1','1999-11-01','2018-01-31', 'Hamburg Port Authority (HPA)'],
                           ['Seemannshoeft', 9.880826468086576, 53.540121739291465, 'NaN','NaN', 628.9, 'NaN', 'Wasserstand [mNN]', '1','1999-11-01','2018-01-31', 'Hamburg Port Authority (HPA)'],
                           ['Blankenese', 9.795825319280944, 53.55772718103615, 'NaN','NaN', 634.8, 'NaN', 'Wasserstand [mNN]', '1','1997-11-01','2018-01-31', 'Hamburg Port Authority (HPA)'],
                           ['Stadersand', 9.52666236789312, 53.62957856537966, 'NaN','NaN', 654.8, 'NaN', 'Wasserstand [mNN]', '1','1997-11-01','2017-12-31','Wasserstrassen- und Schifffahrtsamt Hamburg'],
                           ['Neu Darchau', 10.888798703369245 , 53.23227663760018, 'NaN','NaN', 536.44,'NaN', 'Abfluss [cbm/s]', '1440','1903-01-01','2017-12-31','Wasserstrassen- und Schifffahrtsamt Magdeburg'],
                           ['Teufelsbrück', 9.84672570087493, 53.54867065750803, 9.852277590134403,53.54415651135168, 630.8, -4.6, 'Strömungsgeschwindigkeit [m/s]', '1','2017-08-01','2017-12-31','Hamburg Port Authority (HPA)'],
                           ['Bunthaus Nord', 10.070858563422394, 53.459033050110946, 10.068323525245862, 53.457956406954665, 609.4, -2.78 , 'Strömungsgeschwindigkeit [m/s]', '1','2017-08-01','2017-12-31','Hamburg Port Authority (HPA)'],
                           ['Bunthaus Sued', 10.066307190136287, 53.45739935273774, 10.066995553787468,53.454832307667736, 609.4, -2.78 , 'Strömungsgeschwindigkeit [m/s]', '1','2017-08-01','2017-12-31','Hamburg Port Authority (HPA)'],
                           ['Rhinplatte Nord', 9.372247025803897 , 53.79658489924948, 'NaN', 'NaN', 676.458, 'NaN', 'Trübung [FNU]', '5','2000-12-21','2017-12-22','Wasserstrassen- und Schifffahrtsamt Hamburg'],
                           ['Pagensand Nord', 9.475241417429705 , 53.711763406654725, 'NaN', 'NaN', 664.665, 'NaN', 'Trübung [FNU]', '5','2001-07-30','2017-12-13','Wasserstrassen- und Schifffahrtsamt Hamburg'],
                           ['Juelsand', 9.569255731082903 , 53.608025941016116, 'NaN', 'NaN', 651.323, 'NaN', 'Trübung [FNU]', '5','2000-12-19','2017-12-31','Wasserstrassen- und Schifffahrtsamt Hamburg'],
                           ['Hanskalbsand',9.67207480438894 , 53.56571128392121, 'NaN', 'NaN', 642.997, 'NaN','Trübung [FNU]', '5', '2009-01-01','2017-12-31','Wasserstrassen- und Schifffahrtsamt Hamburg'],
                           ['Geesthacht',10.398624019514566 , 53.410144675229176, 'NaN', 'NaN', 581, 'NaN','Trübung [FNU]', '5', '2016-10-26','2017-01-02','BfG'],
                           ['2765-PH', 9.901389973410712 , 53.53532296756288, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2766-PH', 9.907072983275926 , 53.53306057870628, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2767-PH', 9.909315461706166 , 53.53424757268747, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2768-PH', 9.907137356073664 , 53.536070184899046, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2769-PH', 9.907572900675344 , 53.53809756773997, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2770-PH', 9.903151208703962 , 53.53728644609773, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2771-PH', 9.902657910756911 , 53.53678699384693, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2772-PH', 9.903036776322175 , 53.535454299085494, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2773-PH', 9.902625020421764 , 53.53524176167585, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                           ['2774-PH', 9.904116616457337 , 53.53516758749099, 'NaN', 'NaN', 626, 'NaN', 'Kernprobe', '0', '2017-05-03','2017-05-03','HPA'],
                          ])

mess_infos = mess_infos.set_index('Station')

In [1]:
# 1. lese dfs0 Abflusszeitreihe 
# 2. Erstelle Häufigkeitsverteilung für Abflüsse
# 3. Nutzereingabe Qt1 und Qt2 
# 4. klassifiziere in Q<Qt1 und Q>Qt2
# for dfs file handling
%matplotlib inline
%matplotlib inline

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import numpy as np
from ipywidgets import widgets
from ipywidgets import Layout, interact
from IPython.display import display


import os
from os import listdir # um Ordnerinhalte aufzulisten
from os.path import join, isdir, isfile

from ipywidgets import Layout, interactive, interact
from ipywidgets import SelectionRangeSlider, Dropdown, ToggleButtons, Checkbox, IntSlider, FloatSlider


import sys
from clr import AddReference
sys.path.append(r"C:\Program Files (x86)\DHI\2019\MIKE SDK\bin")
AddReference("DHI.Generic.MikeZero.DFS");
import DHI.Generic.MikeZero.DFS as DFS

import clr
clr.AddReference("DHI.Generic.MikeZero.DFS");
clr.AddReference("DHI.Generic.MikeZero.EUM");
clr.AddReference("System");
import System
from System import Array
from System.Diagnostics import Stopwatch
from DHI.Generic.MikeZero import eumUnit, eumItem, eumQuantity
from DHI.Generic.MikeZero.DFS import *
from DHI.Generic.MikeZero.DFS.dfs123 import *

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot');
%matplotlib inline
lo = Layout(flex='1 1 0%', width='99%') # Layout (Breite etc der widgets)

from pandas.plotting import register_matplotlib_converters

# GUI Layout Vorformattierung
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='97%')

def read_dfs0(infile):
    """
    A script loading a dfs0 file and calculating the average.
    """
    # Load the file
    dfs0File  = DfsFileFactory.DfsGenericOpen(infile);

    # Read times and data
    # try und except um ggf falsches Datumsformat abzufangen 
   
    try:
        start_datetime = pd.to_datetime(str(dfs0File.FileInfo.TimeAxis.StartDateTime),
                                    format='%d.%m.%Y %H:%M:%S')
    except:
        pass
        
    try:
        start_datetime = pd.to_datetime(str(dfs0File.FileInfo.TimeAxis.StartDateTime),
                                    format='%d/%m/%y %H:%M:%S')
    except:
        start_datetime = pd.to_datetime(str(dfs0File.FileInfo.TimeAxis.StartDateTime))
        
    datimes = [] # liste mit Datetimes aller Zeitschritte
    t = []     # Zeitschritte in sekunden
    data = []  # Daten von allen items
    for ii in range(dfs0File.ItemInfo.Count):
        data.append([])
    for it in range(dfs0File.FileInfo.TimeAxis.NumberOfTimeSteps):
        for ii in range(dfs0File.ItemInfo.Count):
            itemData = dfs0File.ReadItemTimeStep(ii+1,it);
            if (ii == 0):
                t.append(itemData.Time);
                datimes.append(start_datetime + pd.Timedelta(itemData.Time, unit='s'))
            data[ii].append(itemData.Data[0]);
    
    # Lese items in dfs Datei   
    nItems = dfs0File.ItemInfo.Count                                    # count items contained in dfs1
    infoItems = [dfs0File.ItemInfo[itm].Name for itm in range(nItems)]  # count items contained in dfs1
    
    dfs0File.Close();
    
    # Wandle um in Dataframe
    df = pd.DataFrame(data=data,index=infoItems).T
    df['time [s]'] = t                # füge Zeitschritte als Spalte hinzu
    df['datetime'] = datimes          # füge datetimes als Spalte hinzu
    df = df.set_index('datetime')     # setzt datetimes als index
    
    return df

def find_folders(sdir):
    """
    finde alle ordner in vorgegebenem ordner
    sdir: Ausgangsordner
    """
    # gehe durch alle Dateien und Ordner in sdir
    slist = [] 
    for f in listdir(sdir):
        fc = join(sdir,f)
        # prüfe ob Ordner
        if isdir(fc):
            # füge Ordner zu liste Hinzu
            slist.append(f)
    return slist

def rem_duplicates(alist):
    """
    Remove duplicates from a list.
    Input: 
        alist: a list of values
    Output:
        uniq: a list with unique values of alist
    """
    # create new list without duplicates and return it
    seen = set()
    uniq = []
    for x in alist:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq


# 1. Lese dfs0 Zeitreihe --------------------------------------------------------------------
fname_Q = 'Neu_Darchau_Q_2008_2017.dfs0'      # Dateiname
path_Q = './demobook_data/'+fname_Q          # füge Pfad und Dateinamen zusammen

df_Q_meas = read_dfs0(path_Q)# Lese dfs0 

# erhalte Dataframe mit Werten für jeden Tag
# Berechne Jahresmittel (für jedes Jahr)
df_Q_meas['dates'] = pd.to_datetime(df_Q_meas.index)
df_Q_meas['Jahresmittel'] = df_Q_meas.groupby(df_Q_meas.dates.dt.year)['Abfluss'].transform('mean')

# Berechne Monatsmittel (global, über alle Jahre sind Monatswerte gemittelt). So wird z.B. sichtbar welche 
# Monate über alle Jahre besonders Abflusswirksam sind 
df_Q_meas['Monatsmittel global'] = df_Q_meas.groupby(df_Q_meas.dates.dt.month)['Abfluss'].transform('mean')

# Berechne Monatsmittel (lokal, Monate zwischen Jahren mit individuellen Werten)
df_Q_meas['Monatsmittel lokal'] = df_Q_meas.groupby([df_Q_meas.dates.dt.month,df_Q_meas.dates.dt.year])['Abfluss'].transform('mean')
#df_Q_meas.head()

# separate dataframes mit genau der Anzahl an Datenpunkten wie in der Mittelung benötigt werden (e.g. Monatlich)
index_year = df_Q_meas.index.year
df_Q_by_year = df_Q_meas['Abfluss'].groupby(index_year).mean()
index_month = df_Q_meas.index.month
df_Q_by_month = df_Q_meas['Abfluss'].groupby(index_month).mean()


# Plotfunktion die auf Veränderung des Sliders reagiert -------------------------------
def fexecute(dat_rng,avg_style,sec_plot):
    plt_bg_col = '#353535'#'#e5e5e5'#'#bababa'#'#353535'#'#444444' # background color for plots
    y_colr = '#42e5f4'    # Farbe für Abfluss
    m_colr = '#ffe649'    # Farbe für Mittel

    m_lw = 2

    fwidth = 17
    fheight = 8
    nBin = 20 # Anzahl der Bins für Histogramm

    variable = 'Abfluss [m$^3$ $s^{-1}$]'
    
    # schneide Daten auf ausgewählte Zeitspanne zu und entferne doppelte Werte in zweitem Schritt
    df_Q_meas_cut = df_Q_meas.loc[dat_rng[0]:dat_rng[1]]
    
    x = df_Q_meas_cut.index
    y = df_Q_meas_cut['Abfluss'].values
    dS = df_Q_meas_cut['Abfluss'] # Tages series/df
    
    m = df_Q_meas_cut['Jahresmittel'].values
    m_uniq = rem_duplicates(m)                   # Liste mit unidentischen werten (für Histogramm)
    mS = df_Q_meas_cut['Jahresmittel']
    
    mm = df_Q_meas_cut['Monatsmittel global'].values  
    mm_uniq = rem_duplicates(mm)
    mmS = df_Q_meas_cut['Monatsmittel global']
    
    mmm = df_Q_meas_cut['Monatsmittel lokal'].values 
    mmm_uniq = rem_duplicates(mmm)
    mmmS = df_Q_meas_cut['Monatsmittel lokal']
    
    
    # Schaue ob und wenn ja welcher zweiter Plot erstellt werden soll 
    # Histogramm --------------------------------------------------------
    if sec_plot == dd_options[1]:
        fig, axs = plt.subplots(2,1, figsize=(fwidth, fheight*2.2),
                        facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .15, wspace=.001)  # horizontal and vertical spacing between subplots
             
        # Format 1. Abbildung
        axs[0].plot(x,y,label='Tageswerte',c=y_colr,marker='.',
                    linewidth=0.5)
        axs[0].set_facecolor(plt_bg_col)
        axs[0].grid(color='w',linestyle=':',linewidth=1)  
        axs[0].set_ylabel(variable)           
        axs[0].set_xlabel('Datum')

        # Format 2. Abbildung (Histogramm)
        axs[1].set_facecolor(plt_bg_col)
        axs[1].grid(color='w',linestyle=':',linewidth=1)  
        axs[1].set_ylabel('Häufigkeit [%]')           
        #axs[1].set_xlabel(variable)
       
    
        # Zusätzliche Daten für beide Plots
        if avg_style == 'Jahresmittel':
            axs[0].plot(x,m,
                    label='Jahresmittel',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
            axs[1].hist([y,m_uniq],bins=nBin,color=['#42e5f4','#ffe649'],label=['Tage','Jahre'],density=True)
            
        elif avg_style == 'Monatsmittel global':
            axs[0].plot(x,mm,
                    label='Monatsmittel global',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)
            axs[1].hist([y,mm],bins=nBin,color=['#42e5f4','#ffe649'],label=['Tage','Monate'],density=True)
            
        elif avg_style == 'Monatsmittel lokal':
            axs[0].plot(x,mmm,
                    label='Monatsmittel lokal',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)
            axs[1].hist([y,mmm],bins=nBin,color=['#42e5f4','#ffe649'],label=['Tage','Monate'],density=True)  
            
        axs[0].legend()
        axs[1].legend()
    
    
    
    # Boxplot ------------------------------------------------------
    if sec_plot == dd_options[2]: 
    # Abbildung erstellen
        fig, axs = plt.subplots(2,1, figsize=(fwidth, fheight*2.2),
                        facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .15, wspace=.001)  # horizontal and vertical spacing between subplots
             
        # Format 1. Abbildung
        axs[0].plot(x,y,label='Tageswerte',c=y_colr,marker='.',
                    linewidth=0.5)
        axs[0].set_facecolor(plt_bg_col)
        axs[0].grid(color='w',linestyle=':',linewidth=1)  
        axs[0].set_ylabel(variable)           
        axs[0].set_xlabel('Datum')

        # Format 2. Abbildung (Histogramm)
        axs[1].set_facecolor('#eaeaea')
        axs[1].grid(color='k',linestyle=':',linewidth=1)  
        axs[1].set_ylabel(variable)           
        #axs[1].set_xlabel('Abfluss [m$^3$ $s^{-1}$]')
       
    
        # Zusätzliche Daten für beide Plots
        if avg_style == 'Jahresmittel':
            axs[0].plot(x,m,
                    label='Jahresmittel',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
            axs[1].boxplot([y,m_uniq],labels=['Tage','Jahre'])
        elif avg_style == 'Monatsmittel global':
            axs[0].plot(x,mm,
                    label='Monatsmittel global',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)
            axs[1].boxplot([y,mm],labels=['Tage','Monate (global)'])
        elif avg_style == 'Monatsmittel lokal':
            axs[0].plot(x,mmm,
                    label='Monatsmittel lokal',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)
            axs[1].boxplot([y,mmm],labels=['Tage','Monate (lokal)'])


        axs[0].legend()
    
    
    
    # Autokorrelation mit zusätzlichem Slider --------------------------
    elif sec_plot == dd_options[3]: 
        # Abbildung erstellen
        fig, axs = plt.subplots(2,1, figsize=(fwidth, fheight*2.2),
                        facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .15, wspace=.001)  # horizontal and vertical spacing between subplots
        
        
        # plotfunktion reagiert auf Änderung des Sliders
        def on_lagvalue_change(lag):
            c_lag =lag['new']
            #print(c_lag)
            # TODO: richtig aktualisieren
            axs[1] = tsaplots.plot_acf(y,lags=c_lag,ax=axs[1],title='')
            return axs[1]
        # erstelle neues widget (slider) mit welchem sich lags in autokorrelation steuern lässt
        lag_slider = IntSlider(value = 30,
                               min = 1,
                               max = 365,
                               step = 1,
                               description='AK Lag',
                               layout=Layout(flex='1 1 0%', width='99%'),
                               continuous_update=False
                              )
        display(lag_slider)
        
        lag_slider.observe(handler=on_lagvalue_change,names='value')
        
        axs[0].plot(x,y,label='Tageswerte',c=y_colr,marker='.',
                    linewidth=0.5)
        # Format 1. Abbildung
        # Zusätzliche Infos für Plot
        if avg_style == 'Jahresmittel':
            axs[0].plot(x,m,
                    label='Jahresmittel',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)            
        elif avg_style == 'Monatsmittel global':
            axs[0].plot(x,mm,
                    label='Monatsmittel global',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
        elif avg_style == 'Monatsmittel lokal':
            axs[0].plot(x,mmm,
                    label='Monatsmittel lokal',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
        axs[0].set_facecolor(plt_bg_col)
        axs[0].grid(color='w',linestyle=':',linewidth=1)  
        axs[0].set_ylabel(variable)           
        axs[0].set_xlabel('Datum')


        # Format 2. Abbildung (Autokorrelation)
        axs[0] = tsaplots.plot_acf(y,lags=30,ax=axs[1],title='')
        axs[1].set_facecolor('#eaeaea')
        
        
    
    # Dekomposition erstelle plot mit drei subplots (trend, saisonalität und noise ) ---------------
    elif sec_plot == dd_options[5]:
        fig, axs = plt.subplots(4,1, figsize=(fwidth, fheight*4.4),
                        facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .15, wspace=.001)  # horizontal and vertical spacing between subplots
             
        # Format 1. Abbildung
        axs[0].plot(x,y,label='Tageswerte',c=y_colr,marker='.',
                    linewidth=0.5)
        axs[0].set_facecolor(plt_bg_col)
        axs[0].grid(color='w',linestyle=':',linewidth=1)  
        axs[0].set_ylabel(variable)           
        axs[0].set_xlabel('Datum')

        # Format 2. Abbildung (Trend und Saisonalitätsplot)
        decomposition = sm.tsa.seasonal_decompose(dS,model='additive')
        axs[1].plot(decomposition.trend,label='Trend Tageswerte')
        axs[2].plot(decomposition.seasonal,label='Saisonalität Tageswerte')
        axs[3].plot(decomposition.resid,label='Rauschen Tageswerte')
        
        # Zusätzliche Daten für beide Plots
        if avg_style == 'Jahresmittel':
            axs[0].plot(x,m,
                    label='Jahresmittel',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)    
            decomposition = sm.tsa.seasonal_decompose(mS,model='additive')
            axs[1].plot(decomposition.trend,label='Jahresmittel')
            axs[2].plot(decomposition.seasonal,label='Jahresmittel')
            axs[3].plot(decomposition.resid,label='Jahresmittel')
        elif avg_style == 'Monatsmittel global':
            axs[0].plot(x,mm,
                    label='Monatsmittel global',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
            decomposition = sm.tsa.seasonal_decompose(mmS,model='additive')
            axs[1].plot(decomposition.trend,label='Monatsmittel global')
            axs[2].plot(decomposition.seasonal,label='Monatsmittel global')
            axs[3].plot(decomposition.resid,label='Monatsmittel global')
        elif avg_style == 'Monatsmittel lokal':
            axs[0].plot(x,mmm,
                    label='Monatsmittel lokal',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)  
            decomposition = sm.tsa.seasonal_decompose(mmmS,model='additive')
            axs[1].plot(decomposition.trend,label='Monatsmittel lokal')
            axs[2].plot(decomposition.seasonal,label='Monatsmittel lokal')
            axs[3].plot(decomposition.resid,label='Monatsmittel lokal')
            
            

        axs[1].set_ylabel('Trend')
        axs[2].set_ylabel('Saisonalität')
        axs[3].set_ylabel('Rauschen')

        axs[0].legend()
        axs[1].legend()
        
        
        
        
    # sonst erstelle nur einen Plot mit Ganglinie -------------------
    elif sec_plot == dd_options[0]: 
        # erstelle Abbildung
        fig, axs = plt.subplots(1,1, figsize=(fwidth, fheight),
                        facecolor='w', edgecolor='k')
        axs.plot(x,y,label='Tageswerte',c=y_colr,marker='.',
                 linewidth=0.5)
        
        # Zusätzliche Infos für Plot
        if avg_style == 'Jahresmittel':
            axs.plot(x,m,
                    label='Jahresmittel',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)            
        elif avg_style == 'Monatsmittel global':
            axs.plot(x,mm,
                    label='Monatsmittel global',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
        elif avg_style == 'Monatsmittel lokal':
            axs.plot(x,mmm,
                    label='Monatsmittel lokal',
                    linestyle='-', linewidth=m_lw,
                    c=m_colr)   
        
        
        axs.set_facecolor(plt_bg_col)
        axs.grid(color='w',linestyle=':',linewidth=1)  
        axs.set_ylabel(variable)           
        axs.set_xlabel('Datum')
        axs.legend()


    
# Slider zur Selektion von Zeiträumen -----------------------------------
dates = pd.date_range('2012-12-31 00:00:00', '2017-12-31 00:00:00',freq='M')
date_options = [(i.strftime('%m/%Y '), i) for i in dates]
dd_options = ['-',
              'Histogram (Wahrscheinlichkeitsdichte)',
              'Boxplot',
              'Autokorrelation',
              'Partielle Autokorrelation',
              'Dekomposition (Trend, Noise/Residual, Saisonalität)',
              'Regression (Q vs Y)']
interactive_Qplot = interactive(fexecute,
                                dat_rng = SelectionRangeSlider(
                                    options=date_options,
                                    index=(0,11),
                                    description='Zeit',
                                    layout=Layout(flex='1 1 0%', width='99%'),
                                    disabled=False,
                                    continuous_update=False
                                ),
                                avg_style = ToggleButtons(
                                    options=['Jahresmittel', 'Monatsmittel global', 'Monatsmittel lokal'],
                                    description='Mittelung:',
                                    disabled=False,
                                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                                    layout=Layout(flex='1 1 0%', width='99%')
                                ),
                                sec_plot = Dropdown(
                                   options=dd_options,  
                                   value = dd_options[0],   # set initial/default vaule to x-position
                                   description='2.Plot',
                                   layout=Layout(flex='1 1 0%', width='99%'))
                               )
# Anzeigen und Formattieren der Anzeige
output = interactive_Qplot.children[-1]
output.layout=box_layout
interactive_Qplot # display everything

FileNotFoundException: File not found: ./demobook_data/Neu_Darchau_Q_2008_2017.dfs0
   at DHI.Generic.MikeZero.DFS.DfsFileFactory.DfsGenericOpen(String filename, DfsFileMode mode, IDfsParameters parameters)

# Simulation runs and documentation 

from the preprocessing we were able to derive Scenarios with relevant (artificial) boundary conditions for simulation. Those are documented in the table below:

<img src="demobook_data/Szenarien.PNG" alt="Szenarien" style="width: 500px;"/>

# Postprocessing

In [4]:
""" Load coordinates from dfsu and save in gdf_coords geodataframe """
def create_element_gdf(dfsufile, ini_crs='epsg:25832', out_crs='epsg:25832'):
    """
    creates a geodataframe from a MIKE (2d) dfsu file which contains 
    x,y,z coordinates and element indices. Coordinate conversion can also be performed.
    -----
    input:
        dfsufile
            str, path and filename of a MIKE (2d) dfsu file
        ini_crs
            str, initial epsg coordinate reference (e.g. epsg:25832 = WGS89 UTM32)
        out_crs
            str, output epsg coordinate reference (e.g. epsg:4326 = Lat/Lon)
    -----
    output:
        dfsu_coord_gdf
            GeoDataFrame, containing x,y,z coordinates and element indices
            via the indices other dfsu data can be accessed later
    -----
    dependencies
    - pydhi (dfsu read)
    - pandas
    - geopandas
    - shapely.geometry
    """
    import pandas as pd
    from pydhi import dfsu 
    import geopandas as gpd
    from shapely.geometry import Point
    
    # load dfsu file data
    dfs = dfsu.dfsu()
    dfs.read(dfsufile)
    
    # create dataframe from dfsu element coordinates
    df = pd.DataFrame(dfs.get_element_coords().T, index=['X','Y','Z'])
    df = df.T
                      
    # create geometry column and convert to geodataframe with coordinate system initialization  
    df["geometry"] = df.apply(lambda x: Point(x["X"], x["Y"]) , axis = 1)
    dfsu_coord_gdf = gpd.GeoDataFrame(df, geometry='geometry', crs={'init': ini_crs})

    # drop now unrequired coordinates
    dfsu_coord_gdf = dfsu_coord_gdf.drop(['X', 'Y'], axis=1)
    # add element column with ids
    dfsu_coord_gdf["Element"] = dfsu_coord_gdf.index
    
    # convert to output coordinate system
    # check if coordinate system should be changed (if input differs from output)
    if ini_crs != out_crs:
        dfsu_coord_gdf = dfsu_coord_gdf.to_crs({'init': out_crs})
        
    return dfsu_coord_gdf

def create_shp_gdf(shp_path, ini_crs='epsg:25832', out_crs='epsg:4326'):
    """
    creates a geodataframe from shapefile. Coordinate conversion can also be performed.
    -----
    input:
        shp_path
            str, path and filename of a *.shp file
        ini_crs
            str, initial epsg coordinate reference (e.g. epsg:25832 = WGS89 UTM32)
        out_crs
            str, output epsg coordinate reference (e.g. epsg:4326 = Lat/Lon)
    output:
        shp_gdf
            GeoDataFrame
    dependencies:
    - geopandas
    """
    import geopandas as gpd
    
    shp_gdf = gpd.read_file(shp_path)
    shp_gdf = shp_gdf.to_crs({'init': ini_crs})
    
    # check if coordinate system should be changed (if input differs from output)
    if ini_crs != out_crs:
        shp_gdf = shp_gdf.to_crs({'init': out_crs})
    
    # display and return result
    #shp_gpd.plot()
    
    return shp_gdf

def create_folium_GJ(fmap, shp_gdf, shp_path, gdf_sjoined, opacity=.3, weight=.2, fill_color='#2b39ff'):
    """
    creates a folium GeoJson
    -----
    input:
        fmap
            str, name of already generated folium map
        shp_gdf
            geodataframe containing geometry column
        opacity
            float, in range 0-1 determining opacity+
        weight
            float, line thickness around shape
        fillColor
            hex, or other color definition
    """
    import folium
    # construct style functions for shapes and add gdfs with shapes to map
    def style_function(feature):
        return {
            'fillOpacity': opacity,
            'weight': weight,
            'fillColor': fill_color
        }
    
    geojson = folium.GeoJson(shp_gdf.geometry,
                   name=shp_path.split('\\')[-1],
                   style_function=style_function).add_to(fmap)
    
    # if elements exist in joined gdf get heights
    if len(gdf_sjoined.Z) != 0:
        minZ = min(gdf_sjoined.Z)
        maxZ = max(gdf_sjoined.Z)
    
        popup = folium.Popup('<b>min.Z: </b>' + str(round(minZ,2)) + ' m \n <b> max.Z: </b>' + str(round(maxZ,2)) + ' m')
        popup.add_to(geojson)

def create_folium_map(shp_path1, shp_path2, 
                      dfsufile,
                      op='intersects',
                      ini_crs1='epsg:25832', 
                      ini_crs2='epsg:25832', 
                      sim_ini_crs='epsg:25832', 
                      out_crs='epsg:4326'):
    """
    Function to create folium map from (widget) input.
    -----
    input:
        shp_path1,2
            str, paths and filenames to one or two shapes
        dfsufile
            str, path and filename of 2d dfsu file to read
        ini_crs1, 2
            str, input etrs coordinates e.g. 'epsg:25832' 
        out_crs
            str, output etrs coordinates , e.g. 'epsg:4326'
    -----
    output:
        fmap
    -----
    dependencies:
    - geopandas as gpd
    - folium
    - from folium.plugins import MeasureControl
    - from folium.plugins import MarkerCluster
    - from folium.plugins import FastMarkerCluster
    - function create_shp_gdf 
    - function create_element_gdf 
    - function create_folium_GJ
    """
    import geopandas as gpd
    import folium
    from folium.plugins import MarkerCluster

    from folium.plugins import MeasureControl
    from folium.plugins import MarkerCluster
    from folium.plugins import FastMarkerCluster
    
    # call function and get returnde gdfs
    shp_gdf1 = create_shp_gdf(shp_path1, ini_crs=ini_crs1, out_crs=out_crs)
    # get element coordinates from simulation
    dfsu_coord_gdf = create_element_gdf(dfsufile, sim_ini_crs, out_crs)
    # then perform sjoin of shape-leftover with element points
    gdf_sjoined = gpd.sjoin(dfsu_coord_gdf, shp_gdf1, how='inner', op=op, lsuffix='left1', rsuffix='right1')
        
    # check if specified shapes differ
    if shp_path1 != shp_path2:
        shp_gdf2 = create_shp_gdf(shp_path2, ini_crs=ini_crs2, out_crs=out_crs)
        gdf_sjoined = gpd.sjoin(gdf_sjoined, shp_gdf2, how='inner', op=op, lsuffix='left2', rsuffix='right2')

        # find centroid (reverse x and y as latitude is first in folium)
        latc = (shp_gdf1.centroid.y[0]+shp_gdf2.centroid[0].y)/2
        lonc = (shp_gdf1.centroid.x[0]+shp_gdf2.centroid[0].x)/2
    else:
        latc = shp_gdf1.centroid.y[0]
        lonc = shp_gdf1.centroid.x[0]
     

    # initialize folium map around centroid coordinates
    fmap = folium.Map(location=[latc,lonc],zoom_start=15,
                      tiles='cartodbpositron',
                      prefer_canvas=True, 
                      control_scale=True)
    # add further background maps
    folium.TileLayer('stamenterrain',attr='Stamen').add_to(fmap)
    folium.TileLayer('openstreetmap',attr='OSM').add_to(fmap)

    # create layer for each 
    create_folium_GJ(fmap, shp_gdf1, shp_path1, gdf_sjoined)
    # only create second layer if specified shapes differ
    if shp_path1 != shp_path2:
        create_folium_GJ(fmap, shp_gdf2, shp_path2, gdf_sjoined, fill_color='#ff0d0d')
    
    # style and plot points (element centers from simulation)   
    element_centers = folium.FeatureGroup(name='Elementmitten')
    for ix, row in gdf_sjoined.iterrows():
        folium.CircleMarker(location=[row.geometry.y, row.geometry.x],
                            radius=1,
                            popup=str(row.Element)+'\n Z: '+str(row.Z),
                            color='#000000').add_to(element_centers)
    element_centers.add_to(fmap)
    
    
    # add a bit of last functionality to map
    folium.LayerControl().add_to(fmap)
    folium.plugins.MeasureControl().add_to(fmap)
    
    # return folium map and the joined geodataframe containing "leftover" element numbers
    return fmap, gdf_sjoined

def create_plotly_cs(dfsufile, dfsufile_layer, gdf_sjoined, exclude_dry=True, itm='Current speed',yr=[0,1.2]):
    """
    create plotly visualization of current speeds at selected elements
    -----
    input:
        dfsufile,
            str, path and filename to area (2d) dfsu file containing surface elevation
        dfsufile_layer
            str, path and filename of dfsufile containing velocities
        exclude_dry
            bool, if True, elements which fall dry are excluded
        gdf_sjoined
            geodataframe, containing intersected elements (created e.g. by create_folium_map)
        itm
            str, item from layer dfsu
        yr
            list, limits of y-axis
    -----
    output:
        fig,
            plotly plot
    -----
    dependencies:
    - sys
    - from toolbox import dfs2df_dict
    - plotly 
    """
    import sys
    sys.path.append(r'C:\Users\clcr\AppData\Local\Continuum\anaconda3\Lib\site-packages\clcr_tools')
    from toolbox import dfs2df_dict
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    
    # get current speeds from layer dfsu file and only selected itm
    df_dict_layer = dfs2df_dict(dfsufile_layer)
    df_itm = df_dict_layer[itm]
    
    # truncate data to selected elements
    df_itm_cut = df_itm[gdf_sjoined.index]
    n_ele_shape = df_itm_cut.shape[1] # number of elements in shapes
    
    # get surface elevation from area file
    df_dict_area = dfs2df_dict(dfsufile) 
    # extract surface elevation from dict
    df_se = df_dict_area['Surface elevation']
    # truncate to selected elements
    df_se_cut = df_se[gdf_sjoined.index]
    # drop values containing nans columnwise if any entry is nan
    df_se_cleaned = df_se_cut.dropna(axis=1,how='any')
    
    """ eliminate elements that fall dry """
    if exclude_dry:
        # finally apply to cs so that no elements which fall dry are included
        df_itm_cut = df_itm[df_se_cleaned.columns]
        n_ele_shape_clean = df_itm_cut.shape[1] # number of elements in shapes after cleaning Nans
        
        # title text for plot
        title_txt = "Geschwindigkeiten für ausgewählte Elemente (trockenfallende exkludiert) und Schicht (ni="+str(n_ele_shape)+" n-wet=" + str(n_ele_shape_clean) + " )"
    else:
        title_txt = "Geschwindigkeiten für ausgewählte Elemente (trockenfallende inkludiert) und Schicht (ni="+str(n_ele_shape) +")"

    # calculate average surface elevation for selected elements
    df_se_cleaned_mean = df_se_cleaned.mean(axis=1)   
    
    # calculate statistics
    mean_layer = df_itm_cut.mean(axis=1)
    max_layer = df_itm_cut.max(axis=1)
    std_layer = df_itm_cut.std(axis=1)

    # gesamtmittel
    mean_all_layer = round(mean_layer.mean(axis=0),2)
    max_all_layer = round(max_layer.max(axis=0),2)
    std_all_layer = round(std_layer.mean(axis=0),2)


    std_plus = mean_layer.values + std_layer.values
    std_minus = mean_layer.values - std_layer.values

    """ plot Ändere zu 3 x 2 grid: oben links wl, oben rechts legende, unten links cs, unten rechts histogram """
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
                        specs=[[{"rowspan": 1}],
                               [{"rowspan": 3}],
                               [None],[None]])

    # plot 1 (top, showing waterlevel)
    fig.add_trace(go.Scatter(x=df_se_cleaned_mean.index, y=df_se_cleaned_mean.values, name="gemittelter Wasserstand",
                         line=dict(color='blue', width=3)),row=1,col=1)
    fig.update_yaxes(title_text="Wasserstand [m.NHN]", row=1, col=1)
    
    # plot 2 (bottom, showing velocities)
    fig.add_trace(go.Scatter(x=mean_layer.index, y=mean_layer.values, name="mittlere Geschwindigkeit: "+str(mean_all_layer)+" m/s)",
                         line=dict(color='black', width=3)),row=2,col=1)
    fig.add_trace(go.Scatter(x=std_layer.index, y=std_plus, name="std+: "+str(std_all_layer)+" m/s)",
                         line_color='lightgray', fill='tonexty'),row=2,col=1)
    fig.add_trace(go.Scatter(x=std_layer.index, y=std_minus, name="std-: "+str(std_all_layer)+" m/s)",
                         line_color='lightgray',fill='tonexty'),row=2,col=1)
    fig.add_trace(go.Scatter(x=max_layer.index, y=max_layer.values, name="max. Geschwindigkeit: "+str(max_all_layer)+" m/s)",
                         line=dict(color='red', width=1, dash='dot')),row=2,col=1)
    fig.update_xaxes(title_text="Datum und Uhrzeit", row=2, col=1)
    fig.update_yaxes(title_text="Strömungsgeschwindigkeit [m/s]", row=2, col=1, range=yr)

    # finalize layout
    fig.update_layout(autosize=True,
                      height=600,
                      title_text=title_txt,
                      legend=dict(x=0.01, y=0.8))

    fig.show()

def create_plotly_cs_hist(dfsufile, dfsufile_layer, gdf_sjoined, exclude_dry=True, itm='Current speed',yr=[0,1.2]):
    """
    create plotly visualization of current speeds at selected elements
    -----
    input:
        dfsufile,
            str, path and filename to area (2d) dfsu file containing surface elevation
        dfsufile_layer
            str, path and filename of dfsufile containing velocities
        exclude_dry
            bool, if True, elements which fall dry are excluded
        gdf_sjoined
            geodataframe, containing intersected elements (created e.g. by create_folium_map)
        itm
            str, item from layer dfsu
        yr
            list, limits of y-axis
    -----
    output:
        fig,
            plotly plot
    -----
    dependencies:
    - sys
    - from toolbox import dfs2df_dict
    - plotly 
    """
    import sys
    sys.path.append(r'C:\Users\clcr\AppData\Local\Continuum\anaconda3\Lib\site-packages\clcr_tools')
    from toolbox import dfs2df_dict
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    
    # get current speeds from layer dfsu file and only selected itm
    df_dict_layer = dfs2df_dict(dfsufile_layer)
    df_itm = df_dict_layer[itm]
    
    # get timedelta=timesteps of cs in minutes
    dt = df_itm.index[1]-df_itm.index[0]
    dt_mins = dt.value/60000000000

    # truncate data to selected elements
    df_itm_cut = df_itm[gdf_sjoined.index]
    n_ele_shape = df_itm_cut.shape[1] # number of elements in shapes
    
    # get surface elevation from area file
    df_dict_area = dfs2df_dict(dfsufile) 
    # extract surface elevation from dict
    df_se = df_dict_area['Surface elevation']
    # truncate to selected elements
    df_se_cut = df_se[gdf_sjoined.index]
    # drop values containing nans columnwise if any entry is nan
    df_se_cleaned = df_se_cut.dropna(axis=1,how='any')
    
    """ eliminate elements that fall dry """
    if exclude_dry:
        # finally apply to cs so that no elements which fall dry are included
        df_itm_cut = df_itm[df_se_cleaned.columns]
        n_ele_shape_clean = df_itm_cut.shape[1] # number of elements in shapes after cleaning Nans
        
        # title text for plot
        title_txt = "Geschwindigkeiten für ausgewählte Elemente (trockenfallende exkludiert) und Schicht (ni="+str(n_ele_shape)+" n-wet=" + str(n_ele_shape_clean) + " )"
    else:
        title_txt = "Geschwindigkeiten für ausgewählte Elemente (trockenfallende inkludiert) und Schicht (ni="+str(n_ele_shape) +")"

    
    # get everything for a histogram
    y2 = df_itm_cut.values
    y2 = y2.flatten('F') 

    # calculate average surface elevation for selected elements
    df_se_cleaned_mean = df_se_cleaned.mean(axis=1)   
    
    # calculate statistics
    mean_layer = df_itm_cut.mean(axis=1)
    max_layer = df_itm_cut.max(axis=1)
    std_layer = df_itm_cut.std(axis=1)

    # gesamtmittel
    mean_all_layer = round(mean_layer.mean(axis=0),2)
    max_all_layer = round(max_layer.max(axis=0),2)
    std_all_layer = round(std_layer.mean(axis=0),2)


    std_plus = mean_layer.values + std_layer.values
    std_minus = mean_layer.values - std_layer.values

    """ plot Ändere zu 4 x 3 """
    fig = make_subplots(rows=4, cols=3, shared_xaxes=True, vertical_spacing=0.1,horizontal_spacing=0.1,
                        specs=[[{"rowspan": 1, "colspan": 2},None,None],
                               [{"rowspan": 3,"colspan": 2},None,{"rowspan": 3}],
                               [None, None, None],
                               [None ,None, None]])

    # plot 1 (top, showing waterlevel)
    fig.add_trace(go.Scatter(x=df_se_cleaned_mean.index, y=df_se_cleaned_mean.values, name="gemittelter Wasserstand",
                         line=dict(color='blue', width=3)),row=1,col=1)
    fig.update_yaxes(title_text="Wasserstand [m.NHN]", row=1, col=1)
    
    # plot 2 (bottom, showing velocities)
    fig.add_trace(go.Scatter(x=mean_layer.index, y=mean_layer.values, name="mittlere Geschwindigkeit: "+str(mean_all_layer)+" m/s)",
                         line=dict(color='black', width=3)),row=2,col=1)
    fig.add_trace(go.Scatter(x=std_layer.index, y=std_plus, name="std+: "+str(std_all_layer)+" m/s)",
                         line_color='lightgray', fill='tonexty'),row=2,col=1)
    fig.add_trace(go.Scatter(x=std_layer.index, y=std_minus, name="std-: "+str(std_all_layer)+" m/s)",
                         line_color='lightgray',fill='tonexty'),row=2,col=1)
    fig.add_trace(go.Scatter(x=max_layer.index, y=max_layer.values, name="max. Geschwindigkeit: "+str(max_all_layer)+" m/s)",
                         line=dict(color='red', width=1, dash='dot')),row=2,col=1)
    fig.update_xaxes(title_text="Datum und Uhrzeit", row=2, col=1)
    fig.update_yaxes(title_text="Strömungsgeschwindigkeit [m/s]", row=2, col=1, range=yr)
    
    # plot 3, horizontal histogram
    # create a histogram with simiar y axis as velocities and x axis with frequency
    fig.add_trace(go.Histogram(y=y2,
                               histnorm='percent', # or 'probability'
                               showlegend=False,
                               marker_color='black',
                               cumulative_enabled=False,
                               ybins=dict(start=0.0,end=yr[1],size=0.1),
                              ),row=2,col=3) 
    fig.update_yaxes(row=2, col=3,range=yr)
    fig.update_xaxes(title_text="Häufigkeit [%] für dt="+str(dt_mins)+" min", row=2, col=3)#,range=[0,50])
    fig.update_traces(marker_line_color='white',marker_line_width=2, opacity=0.6, row=2, col=3)


    # finalize layout
    fig.update_layout(autosize=True,
                      height=800,
                      #width=900,
                      title_text=title_txt,
                      legend=dict(x=0.72, y=1))

    fig.show()

In [5]:
from ipywidgets import Layout, Button, Box, VBox, Dropdown, Text, Output, Checkbox
import glob

""" create inputs for options (e.g. choose from available sims)"""
# options for shapes
shape_path = r'demobook_data\shapes\shapefiles_etrs89_32N\\'
shapefile_options = glob.glob(shape_path+'**.shp')
shapefile_options = [shpf.split('\\')[-1] for shpf in shapefile_options]

# options for simulation folders (isolate only last folder) \\ at the end of simfold_path required
simfold_path = r'demobook_data\analyse_stroemungsatlas\\'
simfold_options = glob.glob(simfold_path+'sz**')
simfold_options = [simf.split('\\')[-1] for simf in simfold_options]



""" Define widgets separately for each row """

# Items flex proportionally to the weight and the left over space around the text
row_0_widgets = [
    Dropdown(description='Shape 1: ', options=shapefile_options, layout=Layout(flex='2 1 auto', width='auto')),
    Text(description='ini_1_crs: ', value='epsg:25832', layout=Layout(flex='1 1 auto', width='auto'))
]

row_1_widgets = [
    Dropdown(description='Shape 2: ', options=shapefile_options, layout=Layout(flex='2 1 auto', width='auto')),
    Text(description='ini_2_crs: ', value='epsg:25832', layout=Layout(flex='1 1 auto', width='auto'))
]

row_2_widgets = [
    Dropdown(description='Sim.: ', options=simfold_options, layout=Layout(flex='1 1 auto', width='auto'))]

row_3_widgets = [
    Text(description='AreaF: ', value='Area_Untersuchungsgebiet.dfsu', layout=Layout(flex='2 1 auto', width='auto')),
    Text(description='Sim.crs: ', value='epsg:25832', layout=Layout(flex='1 1 auto', width='auto'))
]

row_4_widgets = [
    Text(description='LayerF: ', value='bottom_Untersuchungsgebiet.dfsu', layout=Layout(flex='1 1 auto', width='auto')),
    Checkbox(value=True, description='Zeige Häufigkeiten', disabled=False, layout=Layout(flex='1 1 auto', width='auto'))]

row_5_widgets = [
    Dropdown(description='op: ', options=['intersects','contains','within'], layout=Layout(flex='1 1 auto', width='auto')),
    Checkbox(value=True, description='Exkludiere trockenfallende', disabled=False, layout=Layout(flex='1 1 auto', width='auto'))
]

# button to execute plot of map and graph
exec_btn = Button(description='Plot!', layout=Layout(flex='1 1 auto', width='98%'))



""" format box layout style for widgets and assign each row of widgets to Vbox"""
box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='98%')

row_0 = Box(children=row_0_widgets, layout=box_layout)
row_1 = Box(children=row_1_widgets, layout=box_layout)
row_2 = Box(children=row_2_widgets, layout=box_layout)
row_3 = Box(children=row_3_widgets, layout=box_layout)
row_4 = Box(children=row_4_widgets, layout=box_layout)
row_5 = Box(children=row_5_widgets, layout=box_layout)

# initialize widget display
display(VBox([row_0, row_1, row_2, row_3, row_4, row_5, exec_btn]))

""" 
define what should happen when button is clicked
- create function for creation of plotly plot
"""
# on button clicked create folium map (fmap)
def on_button_clicked(b):
    # get inputs from widgets from items (e.g. row_0_widgets) contain a list of their widgets.
    # e.g. by row_0_widgets[0].value the current value of the widget    
    from IPython.display import clear_output
    # clear output but wait with it until a new output has been created
    clear_output(wait=True)
    
    # re-initialize display of widgets
    display(VBox([row_0, row_1, row_2, row_3, row_4, row_5, exec_btn]))

    
    # get dfsufilename and path
    dfsufile = simfold_path + row_2_widgets[0].value + '\\' + row_3_widgets[0].value
    
    # create and display map and get leftover coordinates after intersection
    fmap, gdf_sjoined = create_folium_map(shp_path1=shape_path+row_0_widgets[0].value, 
                             shp_path2=shape_path+row_1_widgets[0].value, 
                             dfsufile=dfsufile, 
                             op = row_5_widgets[0].value,
                             ini_crs1=row_0_widgets[1].value, 
                             ini_crs2=row_1_widgets[1].value, 
                             sim_ini_crs=row_3_widgets[1].value, 
                             out_crs='epsg:4326') # fixed for folium for now
    display(fmap)
    
    """ now create plotly plot from elements in gdf_sjoined """
    # function call to create plotly
    dfsufile_layer = simfold_path + row_2_widgets[0].value + '\\' + row_4_widgets[0].value

    # check if histogram should be showed as separate subplot
    if row_4_widgets[1].value:
        create_plotly_cs_hist(dfsufile, dfsufile_layer, gdf_sjoined, exclude_dry=row_5_widgets[1].value, 
                              itm='Current speed',yr=[0,1.2])
    else:
        create_plotly_cs(dfsufile, dfsufile_layer, gdf_sjoined, exclude_dry=row_5_widgets[1].value, 
                         itm='Current speed',yr=[0,1.2])       
    
exec_btn.on_click(on_button_clicked)




# Theoretical considerations

If the client asks: 
1. How does a velocity profile even look over the depth of a river
2. and on which parameters does it depend?
3. and compared to the depth averaged velocities, how high will the surface velocity be?

We picked a textbook analytical/empirical solution given by the underlying equation:

$v(z) = 2.5 \cdot \sqrt{\frac{\tau_0}{\rho}} \cdot \ln(\frac{z}{z_1})$

and are able to explore the theoretical vertical velocity profile and its dependency on underlying parameters interactively.


In [6]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot');

#from statsmodels.graphics import tsaplots # für korrelation, dekomposition etc
#import statsmodels.api as sm # für decomposition
from ipywidgets import SelectionRangeSlider, Dropdown, ToggleButtons, Checkbox, IntSlider, FloatSlider
from ipywidgets import Layout, interactive, interact
lo = Layout(flex='1 1 0%', width='99%') # Layout (Breite etc der widgets)

def update_vplot(wl,tau_0,rho,z1):
    z_all = np.linspace(0.0001,wl,wl*20) # geodätische Höhe von Sohle

    # löse für alle höhen/tiefen
    v = [2.5*np.sqrt(tau_0/rho)*np.log10(z/z1) for z in z_all]
    vm = np.mean(v) # mittel
    # finde schnittpunkt 
    vi = 100
    for idx,vx in enumerate(v):
        if vi > abs(vx-vm):
            vi = abs(vx-vm)
            ix = idx
    zi = z_all[ix]
    
    # stelle grafisch dar
    fig, axs = plt.subplots(figsize=(16,10))
    axs.scatter(v,z_all,c=v,cmap='rainbow')
    axs.plot(v,z_all,linewidth=0.7,c='k',alpha=0.7)
    # plotte noch tiefenmittel und median
    axs.plot([vm,vm],[z_all[0],z_all[-1]], linewidth=0.5,c='grey',label='depth averaged')
    # plotte schnittpunkt tiefenmittel mit Geschwindigkeitsprofil 
    # plotte rechte achse wie line aber normiert von 0-1(00)
    
    # markiere flächig was über mittel liegt
    plt.hlines(xmin=vm,xmax=v[-1],y=z_all[-1])
    
    # plotte box mit abstand v[-1] zu vm (absolut und relativ)
    # Fehlerbox
    bbox_props = dict(boxstyle="square", fc='w', ec="k", lw=1)
    axs.text(0.1, 0.93, "|v$_{surf}$- v$_m$| = "+str(round(abs(v[-1]-vm),4))+" m/s \n"+
             "v$_{surf}$- v$_m$/v$_{surf}$ = "+str(  round(  (v[-1]-vm)/v[-1]*100)) +"%\n"+
             "z$_i$ = "+str(round(zi,2))+" m \n"+
             "z$_i$/z$_{max}$ = "+str(round(zi/z_all[-1]*100,2))+" % \n",
                        ha="left", va="top", rotation=0, size=15,
                        color='k', transform=axs.transAxes,
                        bbox=bbox_props);
    plt.xlabel('v [m/s]')
    plt.ylabel('z [m]')
    plt.title('Tau$_0$: '+str(tau_0)+' rho: '
              +str(rho)+ ' kg/m$^3$ friction coeff: '+str(z1))
    #plt.grid()
    axs.set_xlim(-0.2,0.5)
    plt.legend()
    plt.show()

    
# sliders
wis_wl = IntSlider(
    value=15,
    min=1,
    max=40,
    step=1,
    description='h [m]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_rho = FloatSlider(
    value=1000,
    min=800,
    max=1300,
    step=10,
    description='rho [kg/m $^3$]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_z1 = FloatSlider(
    value=0.001,
    min=0.001,
    max=1,
    step=0.001,
    description='z$_1$ [kg/m$^3$]', # friction coefficient
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_tau = FloatSlider(
    value=1,
    min=0.001,
    max=3,
    step=0.001,
    description='Tau $_0$ [F/L$^2$]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)

interact(update_vplot,wl=wis_wl, tau_0 = wfs_tau, rho = wfs_rho, z1 = wfs_z1);


interactive(children=(IntSlider(value=15, continuous_update=False, description='h [m]', layout=Layout(flex='1 …